In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
import statsmodels.api as sm
from statsmodels.formula.api import glm, ols

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import math

In [10]:
dictionary=pd.read_csv("dictionary.csv")

with open("IMDB_dftouse_dict.json", "r") as fd:
    IMDB = json.load(fd)
IMDB_df = pd.DataFrame(IMDB)

In [18]:
valences = {word: valence for word, valence in zip(dictionary.words.values,dictionary.valences.values)}

In [8]:
dictionary.head()

,valences,words
0,-2,abandon
1,-2,abandoned
2,-2,abandons
3,-2,abducted
4,-2,abduction


In [9]:
IMDB_df.head()

,movie_id,movie_name,positive,stars,text,url
0,10027,Titanic,True,7,"Sure, Titanic was a good movie, the first time...",http://www.imdb.com/title/tt0120338/usercommen...
1,10028,Titanic,True,10,When I saw this movie I was stunned by what a ...,http://www.imdb.com/title/tt0120338/usercommen...
2,10029,Titanic,True,10,Why do people bitch about this movie and not a...,http://www.imdb.com/title/tt0120338/usercommen...
3,10030,Titanic,True,10,"What's inexplicable? Firstly, the hatred towar...",http://www.imdb.com/title/tt0120338/usercommen...
4,10031,Titanic,True,10,"Previously, I wrote that I loved ""Titanic"", cr...",http://www.imdb.com/title/tt0120338/usercommen...


In [105]:
from sklearn.feature_extraction import text
stopwords = text.ENGLISH_STOP_WORDS
punctuation = list('.,;:!?()[]{}`''\"@#$%^&*+-|-=~_')

In [106]:
def getValenceInfo(text, valenceDict):
    total_words = len(text.split())
    total, count_relevant = 0, 0
    for word in text.split():
        if word in valenceDict.keys():
            count_relevant += 1
            total += valenceDict[word]
    # return total valence, average valence, length of review in words, fraction of contributing words
    return total, 1.*total/count_relevant, total_words, 1.*count_relevant / total_words

def removeStopWords(text, stopwords):
    new_text = ""
    for word in text.split():
        if word not in stopwords:
            while word[-1] in punctuation:
                word = word[:len(word)-1]
            new_text += word + ' '
    return new_text

In [108]:
nostop = removeStopWords(IMDB_df[IMDB_df.movie_id == 10027].text[0], stopwords).lower()
nostop

u"sure titanic good movie time it really second time opinion film definetly change the time movie underlying love-story think ooh romantic the second time (and i think this just annoying just sit watching movie thinking when d**n ship going sink and impressive times the acting film bad definetly great either was i glad dicaprio did win oscar film i mean does think is anthony hopkins denzel washington he does 1 half-good movie won't film $20 million and suprised hardly films it but about eyes worst character film kate winslet's performance hand wonderful i tink director talented film magnitude together there lesson learned movie love-stories is filmmakers shouldn't try add crummy romance single movie out possible 100 i film mere 71 "

In [109]:
getValenceInfo(nostop, happiness)

(38.020792408535023, 0.37644348919341608, 120, 0.8416666666666667)

In [104]:
# Let's use the labMT dictionary
url = 'http://www.plosone.org/article/fetchSingleRepresentation.action?uri=info:doi/10.1371/journal.pone.0026752.s001'
labmt = pd.read_csv(url, skiprows=2, sep='\t', index_col=0)
 
average = labmt.happiness_average.mean()
happiness = (labmt.happiness_average - average).to_dict()
 
def score(text):
    words = text.split()
    return sum([happiness.get(word.lower(), 0.0) for word in words]) / len(words)


In [65]:
labmt.head()

,happiness_rank,happiness_average,happiness_standard_deviation,twitter_rank,google_rank,nyt_rank,lyrics_rank
word,,,,,,,
laughter,1,8.50,0.9313,3600,--,--,1728
happiness,2,8.44,0.9723,1853,2458,--,1230
love,3,8.42,1.1082,25,317,328,23
happy,4,8.30,0.9949,65,1372,1313,375
laughed,5,8.26,1.1572,3334,3542,--,2332


In [66]:
IMDB_df[IMDB_df.movie_id == 10027]['text'][0]

u"Sure, Titanic was a good movie, the first time you see it, but you really should see it a second time and your opinion of the film will definetly change. The first time you see the movie you see the underlying love-story and think: ooh, how romantic. The second time (and I am not the only one to think this) it is just annoying and you just sit there watching the movie thinking, When is this d**n ship going to sink??? And even this is not as impressive when you see it several times. The acting in this film is not bad, but definetly not great either. Was I glad DiCaprio did not win an oscar for that film, I mean who does he think he is, Anthony Hopkins or Denzel Washington? He does 1 half-good movie and won't do a film for less than $20 million. And then everyone is suprised that there are hardly any films with him in it. But enough about, in my eyes, the worst character of the film. Kate Winslet's performance on the other hand was wonderful. I also tink that the director is very talen

In [69]:
getValenceInfo(IMDB_df[IMDB_df.movie_id == 10027]['text'][0],happiness)

(10.386857757785162, 0.057704765321028674, 244, 0.7377049180327869)